In [2]:
import wicked as w
import numpy as np
import re
from fractions import Fraction
from string import Template

\begin{align}
t_a^{i,(n)}(s) &= [f_a^{i,(n)}+\sum_{ux}^{\mathbf{A}}\Delta_u^x t_{ax}^{iu,(n)}\gamma_u^x]\frac{1-e^{-s(\Delta_a^i)^2}}{\Delta_a^i}\\
t_{ab}^{ij,(n)}(s) &= v_{ab}^{ij,(n)}\frac{1-e^{-s(\Delta_{ab}^{ij})^2}}{\Delta_{ab}^{ij}},
\end{align}
where $n=1,2$, and $f_a^{i,(2)}$ and $v_{ab}^{ij,(2)}$ are one- and two-body non-diagonal elements of $\frac{1}{2}[\tilde{H}^{(1)}(s),\hat{A}^{(1)}(s)]$.


We have further that 
\begin{align}
E_{\mathrm{u}}^{(2)}(s)&=\langle\Psi_0|[\tilde{H}^{(1)}(s),\hat{T}^{(1)}(s)]|\Psi_0\rangle\\
E_{\mathrm{u}}^{(3)}(s)&=\langle\Psi_0|[\tilde{H}^{(1)}(s),\hat{T}^{(2)}(s)]|\Psi_0\rangle + \langle\Psi_0|[\tilde{H}^{(2)}(s),\hat{T}^{(1)}(s)]|\Psi_0\rangle
\end{align}
where
\begin{align}
\tilde{H}^{(2)} &= \bar{H}^{(2)} - \frac{1}{6}[[[\hat{H}^{(0)},\hat{A}^{(1)}],\hat{A}^{(1)}],\hat{A}^{(1)}]\\
&=[\hat{H}^{(0)},\hat{A}^{(2)}]+\frac{1}{2}[\tilde{H}^{(1)},\hat{A}^{(1)}] - \frac{1}{6}[[\hat{H}^{(0)},\hat{A}^{(1)}],\hat{A}^{(1)}]\\
&=[\hat{H}^{(0)},\hat{T}^{(2)}]+\frac{1}{2}[\tilde{H}^{(1)},\hat{T}^{(1)}] - \frac{1}{6}\hat{C}^{(2)} + \mathrm{h.c.},
\end{align}
where
\begin{equation}
\hat{C}^{(2)}=[([\hat{H}^{(0)},\hat{T}^{(1)}] + \mathrm{h.c.}),\hat{T}^{(1)}].
\end{equation}


Therefore, the general strategy in implementing DSRG-MRPT3 is
1. Compute non-diagonal elements of $\hat{C}^{(2)}$
2. Compute non-diagonal elements of $[\tilde{H}^{(1)}(s),\hat{T}^{(1)}(s)]$
3. Use the above to compute $\hat{T}^{(2)}$
4. Compute non-diagonal elements of $[\hat{H}^{(0)},\hat{T}^{(2)}]$
5. Assemble $\tilde{H}^{(2)}$
6. Compute the energy contributions.

These can be achieved by these subroutines
1. H1, T1/T2 -> O1/O2
2. H1/H2, T1/T2 -> O1/O2

Here we first generate the MRPT2 energy and the AAAA block of $\bar{H}$

In [3]:
def split_single_tensor(tensor):
    """
    Split a single expression or string of the form
    >>> H^{a0,a3}_{a1,a2}
    into a tuple containing its label, the upper, and lower indices, i.e.,
    >>> ('H', [a0, a3], [a1, a2])
    If the label is 'F' or 'V', the upper and lower indices are swapped.
    """
    tensor = str(tensor)
    label, indices = str(tensor).split('^{')
    upper = indices.split('}_')[0].split(',')
    lower = indices.split('_{')[1].split('}')[0].split(',')
    if label in ['F', 'V']: 
        upper, lower = lower, upper
    return label, upper, lower

def get_unique_tensor_indices(tensor, unused_indices, index_dict):
    label, upper, lower = tensor
    indstr = ''
    for i in upper+lower:
        if index_dict.get(i):
            indstr += index_dict[i]
        else:
            index = unused_indices[i[0]].pop(0)
            index_dict[i] = index
            indstr += index
    
    return indstr

def get_tensor_slice(tensor):
    if tensor[0] in ['gamma1', 'eta1', 'lambda2', 'lambda3']:
        return tensor[0]
    else:
        return tensor[0] + "['" + ''.join([i[0] for i in tensor[1]]) + ''.join([i[0] for i in tensor[2]]) + "']"

def get_lhs_tensor_name(tensor):
    return tensor[0] + ''.join([i[0] for i in tensor[1]]) + ''.join([i[0] for i in tensor[2]])

def get_factor(expression):
    factor = str(expression).split('+=')[-1].split(' ')[1]
    try:
        return float(Fraction(factor))
    except ValueError:
        if factor == '-':
            return -1.0
        else:
            return 1.0

def compile_einsum(expression):
    unused_indices = {'a':list('uvwxyz'), 'c':list('ijk'), 'v':list('abc')}
    index_dict = {}

    lhs = expression.lhs()
    rhs = expression.rhs()

    factor = get_factor(expression)

    exstr = f"+= {factor:+.8f} * np.einsum('"
    tenstr = ''

    for i in str(rhs).split(' '):
        _ = split_single_tensor(i)
        tenstr += get_tensor_slice(_) + ', '
        exstr += get_unique_tensor_indices(_, unused_indices, index_dict)
        exstr += ','

    exstr = exstr[:-1] + '->'
    tenstr += "optimize='optimal')"

    _ = split_single_tensor(lhs)
    if (_[1] != ['']):
        left = get_lhs_tensor_name(_)
        exstr += get_unique_tensor_indices(_, unused_indices, index_dict)
    else:
        left = _[0]
    exstr += "', " + tenstr

    return left + ' ' + exstr

def make_code(mbeq):
    code = ''
    for i in mbeq:
        code += compile_einsum(i) + '\n'
    return code

In [7]:
w.reset_space()
w.add_space("c", "fermion", "occupied", list('ijk'))
w.add_space("a", "fermion", "general", list('uvwxyz'))
w.add_space("v", "fermion", "unoccupied", list('abc'))
wt = w.WickTheorem()

Eref = w.op("E_0",[''])
F0 = w.op("F",['c+ c','a+ a', 'v+ v'])
F1 = w.utils.gen_op('F',1,'cav','cav',diagonal=False)
V1 = w.utils.gen_op('V',2,'cav','cav')
H0op = Eref + F0
H1op = F1 + V1
Hop = H0op + H1op

T1op = w.utils.gen_op('T1',1,'av','ca',diagonal=False)
T2op = w.utils.gen_op('T2',2,'av','ca',diagonal=False)
Top = T1op + T2op

HT = w.commutator(H1op, Top)
HTexpr = wt.contract(HT, 0, 4)
HTeq = HTexpr.to_manybody_equation("H")

E2op = w.commutator(Hop, Top)
E2expr = wt.contract(E2op, 0, 0)
E2eq = E2expr.to_manybody_equation("E2")

In [8]:
print(make_code(E2eq['|']))

E2 += +1.00000000 * np.einsum('iu,iv,vu->', F['ca'], T1['ca'], eta1, optimize='optimal')
E2 += -0.50000000 * np.einsum('iu,ivwx,wxuv->', F['ca'], T2['caaa'], lambda2, optimize='optimal')
E2 += +1.00000000 * np.einsum('ia,ia->', F['cv'], T1['cv'], optimize='optimal')
E2 += +1.00000000 * np.einsum('ua,va,uv->', F['av'], T1['av'], gamma1, optimize='optimal')
E2 += -0.50000000 * np.einsum('ua,vwxa,uxvw->', F['av'], T2['aaav'], lambda2, optimize='optimal')
E2 += -0.50000000 * np.einsum('iu,ivwx,uvwx->', T1['ca'], V['caaa'], lambda2, optimize='optimal')
E2 += -0.50000000 * np.einsum('ua,vwxa,vwux->', T1['av'], V['aaav'], lambda2, optimize='optimal')
E2 += +0.25000000 * np.einsum('ijuv,ijwx,vx,uw->', T2['ccaa'], V['ccaa'], eta1, eta1, optimize='optimal')
E2 += +0.12500000 * np.einsum('ijuv,ijwx,uvwx->', T2['ccaa'], V['ccaa'], lambda2, optimize='optimal')
E2 += +0.50000000 * np.einsum('iuvw,ixyz,wz,vy,xu->', T2['caaa'], V['caaa'], eta1, eta1, gamma1, optimize='optimal')
E2 += +1.00000000 * np.

In [9]:
print(make_code(HTeq['a|a']))

Haa += -1.00000000 * np.einsum('iu,iv->uv', F['ca'], T1['ca'], optimize='optimal')
Haa += -1.00000000 * np.einsum('iu,ivwx,xu->vw', F['ca'], T2['caaa'], eta1, optimize='optimal')
Haa += -1.00000000 * np.einsum('ia,iuva->uv', F['cv'], T2['caav'], optimize='optimal')
Haa += +1.00000000 * np.einsum('ua,va->vu', F['av'], T1['av'], optimize='optimal')
Haa += +1.00000000 * np.einsum('ua,vwxa,uw->vx', F['av'], T2['aaav'], gamma1, optimize='optimal')
Haa += -1.00000000 * np.einsum('iu,ivwx,ux->wv', T1['ca'], V['caaa'], eta1, optimize='optimal')
Haa += -1.00000000 * np.einsum('ia,iuva->vu', T1['cv'], V['caav'], optimize='optimal')
Haa += +1.00000000 * np.einsum('ua,vwxa,wu->xv', T1['av'], V['aaav'], gamma1, optimize='optimal')
Haa += -0.50000000 * np.einsum('ijuv,ijwx,vx->wu', T2['ccaa'], V['ccaa'], eta1, optimize='optimal')
Haa += +0.50000000 * np.einsum('iuvw,ixyz,wxyz->uv', T2['caaa'], V['caaa'], lambda2, optimize='optimal')
Haa += -1.00000000 * np.einsum('iuvw,ixyz,wz,xu->yv', T2['caaa'], V

In [10]:
print(make_code(HTeq['aa|aa']))

Haaaa += -0.50000000 * np.einsum('iu,ivwx->uvwx', F['ca'], T2['caaa'], optimize='optimal')
Haaaa += -0.50000000 * np.einsum('ua,vwxa->vwux', F['av'], T2['aaav'], optimize='optimal')
Haaaa += -0.50000000 * np.einsum('iu,ivwx->wxuv', T1['ca'], V['caaa'], optimize='optimal')
Haaaa += -0.50000000 * np.einsum('ua,vwxa->uxvw', T1['av'], V['aaav'], optimize='optimal')
Haaaa += +0.12500000 * np.einsum('ijuv,ijwx->wxuv', T2['ccaa'], V['ccaa'], optimize='optimal')
Haaaa += +0.25000000 * np.einsum('iuvw,ixyz,xu->yzvw', T2['caaa'], V['caaa'], gamma1, optimize='optimal')
Haaaa += +1.00000000 * np.einsum('iuvw,ixyz,wz->uyvx', T2['caaa'], V['caaa'], eta1, optimize='optimal')
Haaaa += +1.00000000 * np.einsum('iuva,iwxa->uxvw', T2['caav'], V['caav'], optimize='optimal')
Haaaa += +1.00000000 * np.einsum('uvwa,xyza,yv->uzwx', T2['aaav'], V['aaav'], gamma1, optimize='optimal')
Haaaa += +0.25000000 * np.einsum('uvwa,xyza,wz->uvxy', T2['aaav'], V['aaav'], eta1, optimize='optimal')
Haaaa += +0.12500000 * np.